# Kerr spacetime

This notebook demonstrates a few capabilities of SageMath in computations regarding Kerr spacetime. The corresponding tools have been developed within the  [SageManifolds](https://sagemanifolds.obspm.fr) project.

*NB:* a version of SageMath at least equal to 8.2 is required to run this notebook:

In [ ]:
version()

First we set up the notebook to display mathematical objects using LaTeX rendering:

In [ ]:
%display latex

and we initialize a time counter for benchmarking:

In [ ]:
import time
comput_time0 = time.perf_counter()

Since some computations are quite heavy, we ask for running them in parallel on 8 threads:

In [ ]:
Parallelism().set(nproc=8)

## Spacetime manifold

We declare the Kerr spacetime (or more precisely the part of it covered by Boyer-Lindquist coordinates) as a 4-dimensional Lorentzian manifold $\mathcal{M}$:

In [ ]:
M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
print(M)

We then introduce the standard <strong>Boyer-Lindquist coordinates</strong> as a chart `BL` (for *Boyer-Lindquist*) on $\mathcal{M}$, via the method `chart()`, the argument of which is a string
(delimited by `r"..."` because of the backslash symbols) expressing the coordinates names, their ranges (the default is $(-\infty,+\infty)$) and their LaTeX symbols:

In [ ]:
BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") 
print(BL); BL

In [ ]:
BL[0], BL[1]

<h2>Metric tensor</h2>

<p>The 2 parameters $m$ and $a$ of the Kerr spacetime are declared as symbolic variables:</p>

In [ ]:
var('m, a', domain='real')

We get the (yet undefined) spacetime metric:

In [ ]:
g = M.metric()

<p>The metric is set by its components in the coordinate frame associated with Boyer-Lindquist coordinates, which is the current manifold's default frame:</p>

In [ ]:
rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2
g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2
g.display()

<p>A matrix view of the components with respect to the manifold's default vector frame:</p>

In [ ]:
g[:]

<p>The list of the non-vanishing components:</p>

In [ ]:
g.display_comp()

<h2>Levi-Civita Connection</h2>

<p>The Levi-Civita connection $\nabla$ associated with $g$:</p>

In [ ]:
nabla = g.connection()
print(nabla)

<p>Let us verify that the covariant derivative of $g$ with respect to $\nabla$ vanishes identically:</p>

In [ ]:
nabla(g) == 0

<p>Another view of the above property:</p>

In [ ]:
nabla(g).display()

<p>The nonzero Christoffel symbols (skipping those that can be deduced by symmetry of the last two indices):</p>

In [ ]:
g.christoffel_symbols_display()

<h2>Killing vectors</h2>
<p>The default vector frame on the spacetime manifold is the coordinate basis associated with Boyer-Lindquist coordinates:</p>

In [ ]:
M.default_frame() is BL.frame()

In [ ]:
BL.frame()

<p>Let us consider the first vector field of this frame:</p>

In [ ]:
xi = BL.frame()[0]
xi

In [ ]:
print(xi)

<p>The 1-form associated to it by metric duality is</p>

In [ ]:
xi_form = xi.down(g)
xi_form.display()

<p>Its covariant derivative is</p>

In [ ]:
nab_xi = nabla(xi_form)
print(nab_xi)
nab_xi.display()

<p>Let us check that the Killing equation is satisfied:</p>

In [ ]:
nab_xi.symmetrize() == 0

<p>Similarly, let us check that $\frac{\partial}{\partial\phi}$ is a Killing vector:</p>

In [ ]:
chi = BL.frame()[3]
chi

In [ ]:
nabla(chi.down(g)).symmetrize() == 0

<h2>Curvature</h2>

<p>The Ricci tensor associated with $g$:</p>

In [ ]:
Ric = g.ricci()
print(Ric)

Let us check that the Kerr metric is a solution of the **vacuum Einstein equation**:

In [ ]:
Ric == 0

<p>Another view of the above property:</p>

In [ ]:
Ric.display()

<p>The Riemann curvature tensor associated with $g$:</p>

In [ ]:
R = g.riemann()
print(R)

<p>Contrary to the Ricci tensor, the Riemann tensor does not vanish; for instance, the component $R^0_{\ \, 123}$ is</p>

In [ ]:
R[0,1,2,3]

<h3>Bianchi identity</h3>

<p>Let us check the Bianchi identity $\nabla_p R^i_{\ \, j kl} + \nabla_k R^i_{\ \, jlp} + \nabla_l R^i_{\ \, jpk} = 0$:</p>

In [ ]:
DR = nabla(R)   # long (takes a while)
print(DR)

In [ ]:
# from __future__ import print_function  # uncomment for SageMath version < 9.0 (Python 2 based)
for i in M.irange():
    for j in M.irange():
        for k in M.irange():
            for l in M.irange():
                for p in M.irange():
                    print(DR[i,j,k,l,p] + DR[i,j,l,p,k] + DR[i,j,p,k,l], end=' ')

<p>If the last sign in the Bianchi identity is changed to minus, the identity does no longer hold:</p>

In [ ]:
DR[0,1,2,3,1] + DR[0,1,3,1,2] + DR[0,1,1,2,3] # should be zero (Bianchi identity)

In [ ]:
DR[0,1,2,3,1] + DR[0,1,3,1,2] - DR[0,1,1,2,3] # note the change of the second + to -

### Kretschmann scalar

The tensor $R^\flat$, of components $R_{abcd} = g_{am} R^m_{\ \, bcd}$:

In [ ]:
dR = R.down(g)
print(dR)

The tensor $R^\sharp$, of components $R^{abcd} = g^{bp} g^{cq} g^{dr} R^a_{\ \, pqr}$:

In [ ]:
uR = R.up(g)
print(uR)

The Kretschmann scalar $K := R^{abcd} R_{abcd}$:

In [ ]:
Kr_scalar = uR['^{abcd}']*dR['_{abcd}']
Kr_scalar.display()

A variant of this expression can be obtained by invoking the `factor()` method on the coordinate function representing the scalar field in the manifold's default chart:

In [ ]:
Kr = Kr_scalar.coord_function()
Kr.factor()

<p>As a check, we can compare Kr to the formula given by R. Conn Henry, <a href="http://iopscience.iop.org/0004-637X/535/1/350/fulltext/">Astrophys. J. <strong>535</strong>, 350 (2000)</a>:</p>

In [ ]:
Kr == 48*m^2*(r^6 - 15*r^4*(a*cos(th))^2 + 15*r^2*(a*cos(th))^4 
              - (a*cos(th))^6) / (r^2+(a*cos(th))^2)^6

<p>The Schwarzschild value of the Kretschmann scalar is recovered by setting $a=0$:</p>

In [ ]:
Kr.expr().subs(a=0)

<p>Let us plot the Kretschmann scalar for $m=1$ and  $a=0.9$:</p>

In [ ]:
K1 = Kr.expr().subs(m=1, a=0.9)
plot3d(K1, (r, 1, 3), (th, 0, pi), axes_labels=['r', 'theta', 'Kr'])

In [ ]:
print("Total elapsed time: {} s".format(time.perf_counter() - comput_time0))